<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.879 - TFM - Área 2 aula 1 - Machine Learning</p>
<p style="margin: 0; text-align:right;">Máster universitario de Ciencia de datos (<i>Data Science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

# Adquisición de datos
En este módulo se pretenden descargar las imágenes de ImageNet de las clases manzana, pera, plátano, limón, naranja y piña, para poder obtener un primer conjunto de imágenes de entrenamiento y de validación.
Por cada clase descargaremos 200 de entrenamiento y 50 de validación. Asegurando que no existe ninguna imagen que estè repetida en la carpeta de entrenamiento y en la de validación.


In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import cv2
import urllib
import os
import shutil

In [2]:
img_rows, img_cols = 32, 32 #number of rows and columns to convert the images to
input_shape = (img_rows, img_cols, 3)#format to store the images (rows, columns,channels) called channels last
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read it into OpenCV format
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image
n_of_training_images=200#the number of training images to use
n_of_validating_images=50#the number of validating images to use

In [ ]:
synsetsIds=['n07739125','n07767847','n07753592','n07749582','n07747607','n07753275']
synsetsNames=['n07739125','n07767847','n07753592','n07749582','n07747607','n07753275']
#synsetsNames=['apple','pear','banana','lemon','orange','pineapple']

for i in range(0,len(synsetsIds)):
    url="http://www.image-net.org/api/text/imagenet.synset.geturls.getmapping?wnid="+synsetsIds[i]
    page = requests.get(url)
    train_path="TensorFlow/workspace/images/train/"+synsetsNames[i]+"/"
    validation_path="TensorFlow/workspace/images/test/"+synsetsNames[i]+"/"
    
    shutil.rmtree(train_path, ignore_errors=True)
    os.makedirs(train_path)
    
    shutil.rmtree(validation_path, ignore_errors=True)
    os.makedirs(validation_path)
        
    #puts the content of the website into the soup variable, each url on a different line
    soup = BeautifulSoup(page.content, 'html.parser')

    str_soup=str(soup)#convert soup to string so it can be split
    type(str_soup)
    split_urls=str_soup.split('\r\n')#split so each url is a different possition on a list
    #Train data:
    print("Downloading training data: "+synsetsNames[i])
    progress=0
    progress_tmp=0
    while True:
    #for progress in range(n_of_training_images):#store all the images on a directory
        if split_urls[progress_tmp] != None:
            try:
                I = url_to_image(split_urls[progress_tmp].split(' ', 1)[1])
                if (len(I.shape))==3: #check if the image has width, length and channels
                    save_path = train_path+str(split_urls[progress_tmp].split(' ', 1)[0])+'.jpg'#create a name of each image
                    cv2.imwrite(save_path,I)
                    progress+=1
                    #print ("Progress:",(progress/n_of_training_images)*100)
            except:
                None
            progress_tmp+=1
            if progress == n_of_training_images:
                break
                
    #Testing data:
    print("Downloading testing data: "+synsetsNames[i])
    progress2=0
    progress_tmp2=0
    while True:
        #for progress in range(n_of_validating_images):#store all the images on a directory
        if not split_urls[progress+progress2] == None:
            try:
                #get images that are different from the ones used for training
                I = url_to_image(split_urls[progress_tmp+progress_tmp2].split(' ', 1)[1])
                if (len(I.shape))==3: #check if the image has width, length and channels
                    save_path = validation_path+str(split_urls[progress_tmp+progress_tmp2].split(' ', 1)[0])+'.jpg'#create a name of each image
                    cv2.imwrite(save_path,I)
                    progress2+=1
                    #print ("Progress:",(progress2/n_of_validating_images)*100)
            except:
                None
            progress_tmp2+=1
            if progress2 == n_of_validating_images:
                break

## Validación:
Validación de que no existen imágenes repetidas en train y test.

In [24]:
synsetsIds=['n07739125','n07767847','n07753592','n07749582','n07747607','n07753275']
synsetsNames=['n07739125','n07767847','n07753592','n07749582','n07747607','n07753275']
#synsetsNames=['apple','pear','banana','lemon','orange','pineapple']

counter=0
for i in range(0,len(synsetsIds)):
    train_path="TensorFlow/workspace/images/train/"+synsetsNames[i]+"/"
    validation_path="TensorFlow/workspace/images/test/"+synsetsNames[i]+"/"
    train_paths=os.listdir(train_path)
    validation_paths=os.listdir(validation_path)
    for file in validation_paths:
        counter+=1
        #print("Checking file:", file)
        if file in train_paths:
            print("Error duplicated image ["+str(file)+"] in train and test for: "+synsetsNames[i])
    print("Finished checking: "+synsetsNames[i])
print("Checked "+str(counter)+" validation images")

Finished checking: n07739125
Finished checking: n07767847
Finished checking: n07753592
Finished checking: n07749582
Finished checking: n07747607
Finished checking: n07753275
Checked 300 validation images


Validación de que no existe una máscara en png y un fichero xml con las bounding boxes por cada imagen en train y test.

In [25]:
synsetsIds=['n07739125','n07767847','n07753592','n07749582','n07747607','n07753275']
synsetsNames=['n07739125','n07767847','n07753592','n07749582','n07747607','n07753275']
#synsetsNames=['apple','pear','banana','lemon','orange','pineapple']

for i in range(0,len(synsetsIds)):
    train_path="TensorFlow/workspace/images/train/"+synsetsNames[i]+"/"
    validation_path="TensorFlow/workspace/images/test/"+synsetsNames[i]+"/"
    xml_annotations_path="TensorFlow/workspace/annotations/bounding_boxes/"+synsetsNames[i]+"/Annotation/"+synsetsNames[i]+"/"
    masks_annotations_path="TensorFlow/workspace/annotations/masks/"
    
    validation_paths=os.listdir(validation_path)
    validation_pathsli=[x.split('.')[0] for x in validation_paths]
    
    train_paths=os.listdir(train_path)
    train_pathsli=[x.split('.')[0] for x in train_paths]
    
    mask_paths=os.listdir(masks_annotations_path)
    mask_pathsli=[x.split('.')[0] for x in mask_paths]
    
    xml_paths=os.listdir(xml_annotations_path)
    xml_pathsli=[x.split('.')[0] for x in xml_paths]
    
    for file in train_pathsli:
        if file+"_color_mask" not in mask_pathsli:
            print("Error not mask for image ["+str(file)+"] in train for: "+synsetsNames[i])
        if file not in xml_pathsli:
            print("Error not xml for image ["+str(file)+"] in train for: "+synsetsNames[i])
            
    for file in validation_pathsli:
        if file+"_color_mask" not in mask_pathsli:
            print("Error not mask fo## Validación:
Validación de que no existen imágenes repetidas en train y test.r image ["+str(file)+"] in test for: "+synsetsNames[i])
        if file not in xml_pathsli:
            print("Error not xml for image ["+str(file)+"] in test for: "+synsetsNames[i])
            
    print("Finished checking: "+synsetsNames[i])
print("Checked "+str(counter)+" validation images")


Finished checking: n07739125
Finished checking: n07767847
Finished checking: n07753592
Finished checking: n07749582
Finished checking: n07747607
Finished checking: n07753275
Checked 300 validation images
